# Generador de los datos de entrenamiento

## Objetivos del análisis
* Extraer el data frame final con los datos preparados para entrenar algoritmos machine learning.

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

%load_ext version_information
%version_information numpy, pandas

Software versions
Python 3.5.3 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython 5.1.0
OS Linux 4.9.16 gentoo x86_64 with debian stretch sid
numpy 1.12.1
pandas 0.19.2
Sat Apr 29 13:34:01 2017 UTC

In [2]:
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt

pd.options.display.float_format = '{:,.3f}'.format

In [3]:
CHUNK_ID = ["Filename", "DataSet", "Table", "Chunk_Number"]
CHUNK_FEATURES = ["Table", "DType", "Chunk_Size", "Mean", "Median", 
                  "Sd", "Skew", "Kurt", "Min", "Max", "Q1", "Q3",
                  "N_Streaks"]
OUT_OPTIONS = ["Block_Size", "Codec", "Filter", "CL"]
TEST_FEATURES = ["CRate", "CSpeed", "DSpeed"]
COLS = ["Filename" , "DataSet", "Chunk_Number"] + CHUNK_FEATURES 
       + OUT_OPTIONS + TEST_FEATURES
IN_TESTS = ['BLZ_CRate', 'BLZ_CSpeed', 'BLZ_DSpeed', 'LZ4_CRate',
            'LZ4_CSpeed', 'LZ4_DSpeed']
IN_USER = ['IN_CR', 'IN_CS', 'IN_DS']

In [4]:
df = pd.read_csv('../data/blosc_test_data_final.csv.gz', sep='\t')
my_df = df[(df.CL != 0) & (df.CRate > 1.1)]

## Extracción de los datos por chunk

In [5]:
%%time
# DATAFRAME WITH DISTINCT CHUNKS
chunks_df = my_df.drop_duplicates(subset=CHUNK_ID)
print("%d rows" % chunks_df.shape[0])
chunk_tests_list = []
# FOR EACH CHUNK
for index, row in chunks_df.iterrows():
    # DATAFRAME WITH CHUNK TESTS
    chunk_tests_list.append(my_df[(my_df.Filename == row["Filename"]) &
                               (my_df.DataSet == row["DataSet"]) &
                               (my_df.Table == row["Table"]) &
                               (my_df.Chunk_Number == row["Chunk_Number"])])

725 rows
CPU times: user 1min 36s, sys: 76 ms, total: 1min 36s
Wall time: 1min 36s


## Selección de opciones para cada chunk

In [6]:
%%time
training_df = pd.DataFrame()
for chunk_test in chunk_tests_list:
    # EXTRACT MAX MIN AND SOME AUX MAX INDICES
    i_max_crate, i_max_c_speed, i_max_d_speed = \
        chunk_test['CRate'].idxmax(), chunk_test['CSpeed'].idxmax(),\
        chunk_test['DSpeed'].idxmax()
    max_crate, max_c_speed, max_d_speed = \
        (chunk_test.ix[i_max_crate]['CRate'], 
         chunk_test.ix[i_max_c_speed]['CSpeed'],
         chunk_test.ix[i_max_d_speed]['DSpeed'])

    min_crate, min_c_speed, min_d_speed = (chunk_test['CRate'].min(),
                                           chunk_test['CSpeed'].min(),
                                           chunk_test['DSpeed'].min())
    # NORMALIZED COLUMNS
    chunk_test = chunk_test.assign(N_CRate=(chunk_test['CRate'] 
                                            - min_crate) 
                                            / (max_crate - min_crate),
                                   N_CSpeed=(chunk_test['CSpeed'] 
                                            - min_c_speed) 
                                            / (max_c_speed - min_c_speed),
                                   N_DSpeed=(chunk_test['DSpeed'] 
                                            - min_d_speed) 
                                            / (max_d_speed - min_d_speed))
    # DISTANCE FUNC COLUMNS
    chunk_test = chunk_test.assign(Distance_1=(chunk_test['N_CRate'] - 1)**2
                                   + (chunk_test['N_CSpeed'] - 1)**2,
                                   Distance_2=(chunk_test['N_CRate'] - 1)**2
                                   + (chunk_test['N_DSpeed'] - 1) ** 2,
                                   Distance_3=(chunk_test['N_CRate'] - 1)**2
                                   + (chunk_test['N_DSpeed'] - 1)**2
                                   + (chunk_test['N_CSpeed'] - 1)**2,
                                   Distance_4=(chunk_test['N_CSpeed'] - 1)**2
                                   + (chunk_test['N_DSpeed'] - 1) ** 2)
    # BALANCED INDICES
    i_balanced_c_speed, i_balanced_d_speed,
    i_balanced, i_balanced_speeds = (chunk_test['Distance_1'].idxmin(),
                                     chunk_test['Distance_2'].idxmin(),
                                     chunk_test['Distance_3'].idxmin(),
                                     chunk_test['Distance_4'].idxmin())
    indices = [i_max_d_speed, i_max_c_speed, i_balanced_speeds, 
               i_max_crate, i_balanced_d_speed, i_balanced_c_speed,
               i_balanced]
    # TYPE FILTER FOR LZ_DATA
    d_type = chunk_test.iloc[0]['DType']
    filter_name = 'noshuffle'
    if 'float' in d_type or 'int' in d_type:
        filter_name = 'shuffle'
    aux = df[(df.CL == 1) & (df.Block_Size == 0) &
             (df.Filter == filter_name) &
             (df.Filename == chunk_test.iloc[0]['Filename']) &
             (df.DataSet == chunk_test.iloc[0]['DataSet']) &
             (df.Table == chunk_test.iloc[0]['Table']) & 
             (df.Chunk_Number == chunk_test.iloc[0]['Chunk_Number'])]
    lz_data = np.append(aux[aux.Codec == 'blosclz'][TEST_FEATURES].values[0],
                        aux[aux.Codec == 'lz4'][TEST_FEATURES].values[0])
    # APPEND ROWS TO TRAINING DATA FRAME
    for i in range(len(indices)):
        in_1, r = divmod((i+1), 4)
        in_2, in_3 = divmod(r, 2)
        training_df = training_df.append(
                        dict(zip(COLS + IN_TESTS + IN_USER,
                        np.append(np.append(chunk_test.ix[indices[i]][COLS].values,
                        lz_data),
                        [in_1, in_2, in_3]))),
                        ignore_index=True)

CPU times: user 3min 6s, sys: 44 ms, total: 3min 6s
Wall time: 3min 6s


## Algunas comprobaciones

In [7]:
print('DISTINCT MAX RATE')
distinct_max_rate = training_df[(training_df.IN_CR == 1) &
                                (training_df.IN_CS == 0) & 
                                (training_df.IN_DS == 0)]\
                    .drop_duplicates(subset=OUT_OPTIONS)
                    [OUT_OPTIONS + TEST_FEATURES]
print('%s rows' % distinct_max_rate.shape[0])
display(distinct_max_rate.head())
print('DISTINCT MAX C.SPEED')
distinct_max_c_speed = training_df[(training_df.IN_CR == 0) & 
                                   (training_df.IN_CS == 1) &
                                   (training_df.IN_DS == 0)]\
                       .drop_duplicates(subset=OUT_OPTIONS)
                       [OUT_OPTIONS + TEST_FEATURES]
print('%s rows' % distinct_max_c_speed.shape[0])
display(distinct_max_c_speed.head())
print('DISTINCT MAX D.SPEED')
distinct_max_d_speed = training_df[(training_df.IN_CR == 0) &
                                   (training_df.IN_CS == 0) &
                                   (training_df.IN_DS == 1)]\
                      .drop_duplicates(subset=OUT_OPTIONS)
                      [OUT_OPTIONS + TEST_FEATURES]
print('%s rows' % distinct_max_d_speed.shape[0])
display(distinct_max_d_speed.head())
print('DISTINCT BALANCED CSPEED')
distinct_balanced_c_speed = training_df[(training_df.IN_CR == 1) &
                                        (training_df.IN_CS == 1) &
                                        (training_df.IN_DS == 0)]\
                            .drop_duplicates(subset=OUT_OPTIONS)
                            [OUT_OPTIONS + TEST_FEATURES]
print('%s rows' % distinct_balanced_c_speed.shape[0])
display(distinct_balanced_c_speed.head())
print('DISTINCT BALANCED DSPEED')
distinct_balanced_d_speed = training_df[(training_df.IN_CR == 1) &
                                        (training_df.IN_CS == 0) &
                                        (training_df.IN_DS == 1)]\
                            .drop_duplicates(subset=OUT_OPTIONS)
                            [OUT_OPTIONS + TEST_FEATURES]
print('%s rows' % distinct_balanced_d_speed.shape[0])
display(distinct_balanced_d_speed.head())
print('DISTINCT BALANCED SPEED')
distinct_balanced_speed = training_df[(training_df.IN_CR == 0) &
                                      (training_df.IN_CS == 1) &
                                      (training_df.IN_DS == 1)]\
                          .drop_duplicates(subset=OUT_OPTIONS)
                          [OUT_OPTIONS + TEST_FEATURES]
print('%s rows' % distinct_balanced_speed.shape[0])
display(distinct_balanced_speed.head())
print('DISTINCT BALANCED')
distinct_balanced = training_df[(training_df.IN_CR == 1) &
                                (training_df.IN_CS == 1) &
                                (training_df.IN_DS == 1)]\
                    .drop_duplicates(subset=OUT_OPTIONS)
                    [OUT_OPTIONS + TEST_FEATURES]
print('%s rows' % distinct_balanced.shape[0])
display(distinct_balanced.head())

DISTINCT MAX RATE
27 rows


,Block_Size,Codec,Filter,CL,CRate,CSpeed,DSpeed
3,"2,048.000",zstd,shuffle,9.000,1.366,0.014,3.320
654,0.000,zstd,shuffle,9.000,49.115,0.066,4.596
661,0.000,zstd,shuffle,8.000,1.212,0.011,1.145
668,0.000,zstd,shuffle,7.000,1.179,0.013,1.050
696,"2,048.000",zstd,bitshuffle,9.000,4.588,0.057,2.761


DISTINCT MAX C.SPEED
131 rows


,Block_Size,Codec,Filter,CL,CRate,CSpeed,DSpeed
1,64.000,lz4,shuffle,3.000,1.244,6.440,8.259
8,16.000,lz4,shuffle,1.000,1.233,6.758,8.311
22,0.000,lz4,shuffle,1.000,1.217,6.533,8.251
29,64.000,lz4,shuffle,1.000,1.225,6.336,8.421
50,64.000,lz4,shuffle,4.000,1.226,6.166,8.820


DISTINCT MAX D.SPEED
189 rows


...

,Block_Size,Codec,Filter,CL,CRate,CSpeed,DSpeed
4,256.000,lz4hc,shuffle,8.000,1.310,0.134,8.219
11,128.000,lz4hc,shuffle,8.000,1.300,0.153,8.513
25,256.000,lz4hc,shuffle,5.000,1.292,0.190,8.783
32,128.000,lz4hc,shuffle,9.000,1.286,0.117,8.351
39,256.000,lz4hc,shuffle,7.000,1.290,0.147,8.150


DISTINCT BALANCED SPEED
157 rows


,Block_Size,Codec,Filter,CL,CRate,CSpeed,DSpeed
2,32.000,lz4,shuffle,3.000,1.239,6.157,8.908
9,16.000,lz4,shuffle,5.000,1.238,6.366,9.139
16,0.000,lz4,shuffle,4.000,1.239,6.501,9.179
23,128.000,lz4,shuffle,2.000,1.233,6.000,9.044
30,64.000,lz4,shuffle,2.000,1.226,6.196,8.831


DISTINCT BALANCED
148 rows


,Block_Size,Codec,Filter,CL,CRate,CSpeed,DSpeed
6,256.000,lz4,shuffle,5.000,1.252,5.722,8.805
13,128.000,lz4,shuffle,6.000,1.250,6.042,9.115
20,128.000,lz4,shuffle,5.000,1.248,6.201,9.488
27,256.000,lz4,shuffle,6.000,1.242,5.558,8.574
34,0.000,lz4,shuffle,8.000,1.241,5.440,8.557


In [8]:
distinct_total = training_df.drop_duplicates(subset=OUT_OPTIONS)
                            [OUT_OPTIONS + TEST_FEATURES]
print('%d distinct options from a total of %d' %
      (distinct_total.shape[0], 1620))
distinct_total_noblock = distinct_total.drop_duplicates(subset=OUT_OPTIONS[1:4])
print('%d distinct options from a total of %d' %
      (distinct_total_noblock.shape[0], 162))
print('Distinct codecs %d' % 
      distinct_total.drop_duplicates(subset=['Codec']).shape[0])
print('Distinct filters %d' %
      distinct_total.drop_duplicates(subset=['Filter']).shape[0])
print('Distinct CL %d' %
      distinct_total.drop_duplicates(subset=['CL']).shape[0])
print('Distinct block sizes %d' %
      distinct_total.drop_duplicates(subset=['Block_Size']).shape[0])
display(distinct_total.describe())

487 distinct options from a total of 1620
96 distinct options from a total of 162
Distinct codecs 5
Distinct filters 3
Distinct CL 9
Distinct block sizes 10


,Block_Size,CL,CRate,CSpeed,DSpeed
count,487.000,487.000,487.000,487.000,487.000
mean,311.737,5.335,327.070,6.583,13.948
std,553.841,2.583,"1,585.596",6.718,11.312
min,0.000,1.000,1.101,0.004,0.474
25%,16.000,3.000,1.268,0.763,8.743
50%,64.000,6.000,4.112,4.860,10.421
75%,256.000,8.000,49.268,9.370,14.523
max,"2,048.000",9.000,"10,645.442",23.848,86.345


Zlib queda descartado dado que nunca es seleccionado como óptimo.

In [9]:
display(training_df[training_df.Codec == 'snappy']
        [IN_USER + TEST_FEATURES + OUT_OPTIONS])

,IN_CR,IN_CS,IN_DS,CRate,CSpeed,DSpeed,Block_Size,Codec,Filter,CL
1681,0.000,1.000,0.000,21.195,20.349,9.824,0.000,snappy,noshuffle,7.000
4291,0.000,0.000,1.000,11.049,7.158,13.803,128.000,snappy,noshuffle,3.000


Snappy ha sido seleccionado en dos ocasiones. Por tanto podríamos considerar que tenemos 488/1080 opciones totales y sin contar el tamaño de bloque 97/108.

In [10]:
print('%d blosclz classes from 270' %
      distinct_total[distinct_total.Codec == 'blosclz'].shape[0])
print('%d lz4 classes from 270' %
      distinct_total[distinct_total.Codec == 'lz4'].shape[0])
print('%d lz4hc classes from 270' %
      distinct_total[distinct_total.Codec == 'lz4hc'].shape[0])
print('%d zstd classes from 270' %
      distinct_total[distinct_total.Codec == 'zstd'].shape[0])

144 blosclz classes from 270
168 lz4 classes from 270
102 lz4hc classes from 270
71 zstd classes from 270


Debido a que Snappy solo es seleccionado en dos ocasiones lo consideraremos como datos atípicos y por tanto los sustituimos por la segunda mejor opción.

In [11]:
# ELIMINAMOS SNAPPY
for i, row in (training_df[training_df.Codec == 'snappy']).iterrows():
    aux = df[(df.Filename == row['Filename']) &
             (df.DataSet == row['DataSet']) &
             (df.Table == row['Table']) & 
             (df.Chunk_Number == row['Chunk_Number']) &
             (df.Codec != 'snappy')]
    i_max_crate, i_max_c_speed, i_max_d_speed = aux['CRate'].idxmax(),
                                                aux['CSpeed'].idxmax(),\
                                                aux['DSpeed'].idxmax()
    max_crate, max_c_speed, max_d_speed = (aux.ix[i_max_crate]['CRate'],
                                           aux.ix[i_max_c_speed]['CSpeed'],
                                           aux.ix[i_max_d_speed]['DSpeed'])

    min_crate, min_c_speed, min_d_speed = (aux['CRate'].min(),
                                           aux['CSpeed'].min(),
                                           aux['DSpeed'].min())
    # NORMALIZED COLUMNS
    aux = aux.assign(N_CRate=(aux['CRate'] - min_crate) 
                     / (max_crate - min_crate),
                     N_CSpeed=(aux['CSpeed'] - min_c_speed) 
                     / (max_c_speed - min_c_speed),
                     N_DSpeed=(aux['DSpeed'] - min_d_speed) 
                     / (max_d_speed - min_d_speed))
    aux['Distance'] = row['IN_CR']*(aux['N_CRate'] - 1)**2
                      + row['IN_DS']*(aux['N_DSpeed'] - 1)**2 
                      + row['IN_CS']*(aux['N_CSpeed'] - 1)**2
    index = aux['Distance'].idxmin()
    training_df.loc[i, TEST_FEATURES + OUT_OPTIONS] = \
        aux.ix[index][TEST_FEATURES + OUT_OPTIONS]

## Tamaño de bloque automático

In [12]:
%%time
count = training_df[training_df.Block_Size == 0].shape[0]
for i, row in training_df.iterrows():
    block = row['Block_Size']
    aux = df[(df.Filename == row['Filename']) &
             (df.DataSet == row['DataSet']) &
             (df.Table == row['Table']) &
             (df.Chunk_Number == row['Chunk_Number']) &
             (df.Codec == row['Codec']) &
             (df.Filter == row['Filter']) &
             (df.CL == row["CL"])]
    crate = aux[aux.Block_Size == 0]['CRate'].values[0]
    auto_block = aux[(aux.CRate == crate) &
                     (aux.Block_Size != 0)]['Block_Size'].values[0]
    if block != 0:
        if auto_block == block:
            count += 1
    else:
        training_df.loc[i, 'Block_Size'] = auto_block

CPU times: user 24min 29s, sys: 284 ms, total: 24min 29s
Wall time: 24min 29s


In [13]:
print("%d from %d --> %d %%" %
      (count, training_df.shape[0], count / training_df.shape[0] * 100))

1372 from 5075 --> 27 %


In [14]:
training_df.drop_duplicates(subset=['Block_Size'])

,BLZ_CRate,BLZ_CSpeed,BLZ_DSpeed,Block_Size,CL,CRate,CSpeed,Chunk_Number,Chunk_Size,Codec,...,Max,Mean,Median,Min,N_Streaks,Q1,Q3,Sd,Skew,Table
0,1.110,3.959,7.963,32.000,1.000,1.108,4.402,1.000,16.000,blosclz,...,27.146,5.762,3.093,-14.914,"101,125.000",-0.990,12.773,8.290,0.555,0.000
1,1.110,3.959,7.963,64.000,3.000,1.244,6.440,1.000,16.000,lz4,...,27.146,5.762,3.093,-14.914,"101,125.000",-0.990,12.773,8.290,0.555,0.000
3,1.110,3.959,7.963,"2,048.000",9.000,1.366,0.014,1.000,16.000,zstd,...,27.146,5.762,3.093,-14.914,"101,125.000",-0.990,12.773,8.290,0.555,0.000
4,1.110,3.959,7.963,256.000,8.000,1.310,0.134,1.000,16.000,lz4hc,...,27.146,5.762,3.093,-14.914,"101,125.000",-0.990,12.773,8.290,0.555,0.000
8,1.110,5.626,9.007,16.000,1.000,1.233,6.758,2.000,16.000,lz4,...,27.775,6.145,4.291,-10.519,"118,220.000",-1.361,14.216,8.504,0.391,0.000
11,1.110,5.626,9.007,128.000,8.000,1.300,0.153,2.000,16.000,lz4hc,...,27.775,6.145,4.291,-10.519,"118,220.000",-1.361,14.216,8.504,0.391,0.000
21,1.100,5.482,9.048,8.000,2.000,1.119,4.288,4.000,16.000,blosclz,...,33.807,6.450,2.096,-12.255,"109,441.000",-1.446,14.703,10.067,0.742,0.000
82,1.072,5.368,9.172,512.000,8.000,1.243,5.329,12.000,16.000,lz4,...,49.183,10.937,7.797,-14.951,"122,250.000",1.882,19.096,11.198,0.655,0.000
654,1.326,3.743,15.588,"1,024.000",9.000,49.115,0.066,1.000,0.738,zstd,...,"211,383.000","46,750.635","42,412.000",0.000,"27,744.000",121.000,"79,434.750","42,964.463",1.123,0.000


## Preparación de inputs para scikit-learn

In [15]:
from sklearn.preprocessing import binarize 
from sklearn.preprocessing import OneHotEncoder
training_df = training_df.assign(
               is_Table=binarize(training_df['Table'].values.reshape(-1,1), 0), 
               is_Columnar=binarize(training_df['Table'].values.reshape(-1,1), 1),
               is_Int=training_df['DType'].str.contains('int').astype(int),
               is_Float=training_df['DType'].str.contains('float').astype(int),
               is_String=(training_df['DType'].str.contains('S') |
                          training_df['DType'].str.contains('U')).astype(int))
import re
def aux_func(s):
    n = int(re.findall('\d+', s)[0])
    isNum = re.findall('int|float', s)
    if len(isNum) > 0:
        return n // 8
    else:
        return n
training_df['Type_Size'] = [aux_func(s) for s in training_df['DType']]

## Preparación de outputs para scikit-learn

In [16]:
training_df = training_df.assign(
                Blosclz=(training_df['Codec'] == 'blosclz').astype(int),
                Lz4=(training_df['Codec'] == 'lz4').astype(int),
                Lz4hc=(training_df['Codec'] == 'lz4hc').astype(int),
                Snappy=(training_df['Codec'] == 'snappy').astype(int),
                Zstd=(training_df['Codec'] == 'zstd').astype(int),
                Noshuffle=(training_df['Filter'] == 'noshuffle').astype(int),
                Shuffle=(training_df['Filter'] == 'shuffle').astype(int),
                Bitshuffle=(training_df['Filter'] == 'bitshuffle').astype(int))
enc_cl = OneHotEncoder()
enc_cl.fit(training_df['CL'].values.reshape(-1, 1))
new_cls = enc_cl.transform(training_df['CL'].values.reshape(-1, 1)).toarray()
enc_block = OneHotEncoder()
enc_block.fit(training_df['Block_Size'].values.reshape(-1, 1))
new_blocks = enc_block.transform(training_df['Block_Size']
                                 .values.reshape(-1, 1)).toarray()
for i in range(9):
    cl_label = 'CL' + str(i+1)
    block_label = 'Block_' + str(2**(i+3))
    training_df[cl_label] = new_cls[:, i]
    training_df[block_label] = new_blocks[:, i]

In [17]:
training_df.to_csv('../data/training_data.csv', sep='\t', index=False)